In [ ]:
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects import pandas2ri
from anndata import AnnData
import scanpy as sc
from os import path

In [ ]:
!pwd
root = "/engelhardt/home/pshrestha/vnngp/"
dpath = path.join(root, "data")

In [ ]:
# Enable the conversion between R and pandas data frames
pandas2ri.activate()

# Load the Seurat and SeuratDisk packages
seurat = rpackages.importr('Seurat')
seurat_disk = rpackages.importr('SeuratDisk')

In [ ]:
# Define the path to your .rds file
rds_file_path = path.join(dpath, "multi_intergrated_seurat_Dec2020.rds")

# Read the Seurat object
seurat_object = robjects.r['readRDS'](rds_file_path)

# Save the Seurat object as an h5Seurat file
h5seurat_file_path = "path/to/temp/file.h5seurat"
seurat_disk.SaveH5Seurat(seurat_object, filename=h5seurat_file_path, overwrite=True)

# Convert the h5Seurat file to h5ad (AnnData)
h5ad_file_path = "path/to/temp/file.h5ad"
seurat_disk.Convert(h5seurat_file_path, dest='h5ad', overwrite=True)

In [ ]:
# Read the h5ad file as an AnnData object
adata = sc.read_h5ad(h5ad_file_path)

# Display the AnnData object
print(adata)